<a href="https://colab.research.google.com/github/StefanoGenettiUniTN/appa-chinquinaria/blob/data-visualization/data_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data visualization

## Data export bollettino APPA
**Date:** 2025-09-30

**Source:** [link](https://bollettino.appa.tn.it/aria/scarica)

In this section we try to visualize data from bollettino APPA from 2023-01-01 to 2023-03-01. Indeed from the website it is possible to download data within maximum 90 days interval.

**Note:** you can select multiple "Stazioni" and multiple "Inquinanti" by pressing Ctrl and select the items.

In [43]:
%%shell
if [ ! -f "appa-export.csv" ]; then
  gdown -q 1JDDMzu7Jo1polnxJ249Tthim5MaVdTpb -O appa-export.csv
fi

In [44]:
import os
import pandas as pd
import requests
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import zipfile
from ipywidgets import interact, widgets
from typing import List, Tuple, Dict, Any, Optional
from datetime import datetime, date

In [45]:
file_path: str = "appa-export.csv"
df: pd.DataFrame = pd.read_csv(file_path, encoding="latin1")

# parse datetime
# replace '24' in 'Ora' with '0' and handle the date increment
# convert Valore to numeric
df['datetime'] = pd.to_datetime(df['Data'] + ' ' + df['Ora'].replace(24, 0).astype(str) + ':00:00')
df.loc[df['Ora'] == 24, 'datetime'] += pd.Timedelta(days=1)
df['Valore'] = pd.to_numeric(df['Valore'], errors="coerce")  # sometimes valore is n.d. with this it is converted to NaN

# unique options
stations: List[str] = df['Stazione'].unique()
pollutants: List[str] = df['Inquinante'].unique()
min_date: pd.Timestamp = df['datetime'].min()
max_date: pd.Timestamp = df['datetime'].max()
min_hour: int = df['Ora'].min()
max_hour: int = df['Ora'].max()

# interactive plot function
def plot_air_quality( selected_stations: List[str],
                      selected_pollutants: List[str],
                      start_date: date,
                      end_date: date,
                      hour_range: Tuple[int, int]) -> None:
    filtered = df[
        (df['Stazione'].isin(selected_stations)) &
        (df['Inquinante'].isin(selected_pollutants)) &
        (df['datetime'] >= pd.to_datetime(start_date)) &
        (df['datetime'] <= pd.to_datetime(end_date)) &
        (df['Ora'] >= hour_range[0]) &
        (df['Ora'] <= hour_range[1])
    ]

    plt.figure(figsize=(12,6))
    for station, group in filtered.groupby("Stazione"):
        for pollutant, sub_group in group.groupby("Inquinante"):
            plt.plot(sub_group["datetime"], sub_group["Valore"], marker='o', label=f"{station} - {pollutant}")

    plt.title("Andamento inquinanti per stazione")
    plt.xlabel("Tempo")
    plt.ylabel("Valore (µg/mc)")
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))  # format datetime
    plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())  # auto-set tick positions
    plt.xticks(rotation=45, ha='right')  # rotate labels
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

# === widgets ==================================================================
station_widget = widgets.SelectMultiple(
    options=stations,
    value=[stations[0]],
    description='Stazioni',
    disabled=False
)

pollutant_widget = widgets.SelectMultiple(
    options=pollutants,
    value=[pollutants[0]],
    description='Inquinanti',
    disabled=False
)

start_date_widget = widgets.DatePicker(
    value=min_date.date(),
    description='Start Date'
)

end_date_widget = widgets.DatePicker(
    value=max_date.date(),
    description='End Date'
)

hour_range_widget = widgets.IntRangeSlider(
    value=[min_hour, max_hour],
    min=0,
    max=24,
    step=1,
    description='Hour Range',
    continuous_update=False,
    layout={'width': '500px'}
)
# ==============================================================================

interact(
    plot_air_quality,
    selected_stations=station_widget,
    selected_pollutants=pollutant_widget,
    start_date=start_date_widget,
    end_date=end_date_widget,
    hour_range=hour_range_widget
)

interactive(children=(SelectMultiple(description='Stazioni', index=(0,), options=('Parco S. Chiara', 'Via Bolz…

<function __main__.plot_air_quality(selected_stations: List[str], selected_pollutants: List[str], start_date: datetime.date, end_date: datetime.date, hour_range: Tuple[int, int]) -> None>

## Data export European Environment Agency

The purpose of this section is to process PM10 data from European Environment Agency (EEA) API. It would be nice to display the map of Italy with each station as a point and connect the stations with graph edges with edge size proportional to the correlation between stations. At the end of the section we align EEA data with APPA.

In [46]:
# get metadata from google drive: csv file with the vocabulary with station data. Can be also downloaded from https://discomap.eea.europa.eu/App/AQViewer/index.html?fqn=Airquality_Dissem.b2g.measurements#
%%shell
if [ ! -f "metadata-vocabulary-italy.csv" ]; then
  gdown -q 1_u0CXivDI1kTpTOcBgOPysFtOXKsUfjs -O metadata-vocabulary-italy.csv
fi

In [47]:
# API parameters
apiUrl: str = "https://eeadmz1-downloads-api-appservice.azurewebsites.net/"
metadata_vocabulary = "metadata-vocabulary-italy.csv"
pollutant_dict: Dict[str, str] = {
    "PM10": "http://dd.eionet.europa.eu/vocabulary/aq/pollutant/5"
}
endpoint: str = "ParquetFile"

api_countries: List[str] = ["IT"]
api_cities: List[str] = []
api_pollutants: List[str] = ["PM10"]
api_dateTimeStart: str = "2023-01-01"
api_dateTimeEnd: str = "2023-03-01"
api_aggregationType: str = "hour"

# EEA zip file with air pollution data
zip_eea_parquet: str = "eea-pm10-italy.zip"
if not os.path.exists(zip_eea_parquet):
  # request body
  request_body: Dict[str, Any] = {
      "countries": api_countries, # List[str]
      "cities": api_cities, # List[str]
      "pollutants": api_pollutants, # List[str]
      "dataset": 2, # int
      "dateTimeStart": f"{api_dateTimeStart}T00:00:00.000Z",  # str
      "dateTimeEnd": f"{api_dateTimeEnd}T23:59:00.000Z",  # str
      "aggregationType": f"{api_aggregationType}",  # str
      "email": None # str
  }

  # a get request to the API
  print("Requesting zip file from API...")
  downloadFile = requests.post(apiUrl+endpoint, json=request_body).content

  # store in local path
  output = open(zip_eea_parquet, 'wb')
  output.write(downloadFile)
  output.close()
  print(f"Downloaded zip file from API and saved to {zip_eea_parquet}")

# extract parquet files from zip archive
eea_parquet_folder: str = "eea-pm10-italy"
if not os.path.exists(eea_parquet_folder):
  os.makedirs(eea_parquet_folder)
with zipfile.ZipFile(zip_eea_parquet, 'r') as zip_ref:
  zip_ref.extractall(eea_parquet_folder)
parquet_files = [os.path.join(f"{eea_parquet_folder}/E1a", f) for f in os.listdir(f"{eea_parquet_folder}/E1a") if f.endswith('.parquet')]
print(f"Found {len(parquet_files)} parquet files")

# convert parquet files to csv
eea_csv = "eea-pm10-italy.csv"
df_list = [pd.read_parquet(pf) for pf in parquet_files]
combined_df = pd.concat(df_list, ignore_index=True)
combined_df.to_csv(eea_csv, index=False)
print(f"CSV saved to {eea_csv}")

# merge the eea data with the metadata vocabularies
eea_csv_with_metadata = "eea-pm10-italy-full.csv"
df_metadata = pd.read_csv(metadata_vocabulary)
df_eea = pd.read_csv(eea_csv)
df_eea['station-id'] = df_eea['Samplingpoint'].str.split("IT/").str[-1]
df_metadata['station-id'] = df_metadata["Sampling Point Id"]
df_merged = pd.merge(df_eea, df_metadata, on="station-id", how="left", indicator=True)
df_unmatched = df_merged[df_merged["_merge"] == "left_only"]
print(f"Unmatched rows saved as unmatched_measurements.csv (total: {len(df_unmatched)})")
# filter relevant columns
columns_to_keep = [
    "station-id",
    "Start",
    "End",
    "Value",
    "Unit",
    "AggType",
    "Country",
    "Air Pollutant",
    "Longitude",
    "Latitude",
    "Altitude",
    "Altitude Unit",
    "Air Quality Station Area",
    "Air Quality Station Type",
    "Municipality",
    "Duration Unit",
    "Cadence Unit"
]
df_filtered = df_merged[columns_to_keep]
df_filtered.to_csv(eea_csv_with_metadata, index=False)
print(f"Merged CSV saved to {eea_csv_with_metadata}")


Found 122 parquet files
CSV saved to eea-pm10-italy.csv
Unmatched rows saved as unmatched_measurements.csv (total: 0)
Merged CSV saved to eea-pm10-italy-full.csv
